In [6]:
import os

import polars as pl

os.chdir("../../../")

# predictables\encoding\src\lagged_mean_encoding\_dynamic_rolling_sum.py
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_sum import (
    DynamicRollingSum,
)
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_mean import (
    DynamicRollingSum,
)

In [10]:
date_fmt_str = "%m/%d/%Y"

lf = pl.scan_csv(
    "/app/predictables/encoding/tests/rolling_date_example.csv"
).with_columns(
    [
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("incr_value").cast(pl.Float64).name.keep(),
        pl.col("month_prior")
        .str.to_date(format=date_fmt_str)
        .cast(pl.Date)
        .name.keep(),
        pl.col("year_prior").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("rolling_sum").cast(pl.Float64).name.keep(),
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).alias("cur_date"),
        pl.col("cat1").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
        pl.col("cat2").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
    ]
)
lf.head().collect()

date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,30_days_prior,60_days_prior,360_days_prior,390_days_prior,sum2,sum_cat1,sum_cat2,sum_catBOTH,cur_date
date,f64,date,date,f64,cat,cat,str,str,str,str,i64,i64,i64,i64,date
2014-01-01,1.0,2013-12-01,2012-12-01,0.0,"""B""","""C""","""12/2/2013""","""11/2/2013""","""1/6/2013""","""12/7/2012""",0,0,0,0,2014-01-01
2014-01-02,1.0,2013-12-02,2012-12-02,0.0,"""A""","""C""","""12/3/2013""","""11/3/2013""","""1/7/2013""","""12/8/2012""",0,0,0,0,2014-01-02
2014-01-03,1.0,2013-12-03,2012-12-03,0.0,"""B""","""D""","""12/4/2013""","""11/4/2013""","""1/8/2013""","""12/9/2012""",0,0,0,0,2014-01-03
2014-01-04,1.0,2013-12-04,2012-12-04,0.0,"""B""","""D""","""12/5/2013""","""11/5/2013""","""1/9/2013""","""12/10/2012""",0,0,0,0,2014-01-04
2014-01-05,1.0,2013-12-05,2012-12-05,0.0,"""B""","""E""","""12/6/2013""","""11/6/2013""","""1/10/2013""","""12/11/2012""",0,0,0,0,2014-01-05


In [17]:
(
    DynamicRollingSum()
    .lf(lf.with_row_index())
    .x_col("incr_value")
    .date_col("date")
    .index_col("index")
    .offset(30)
    .window(30)
).run().collect()

index,date,rolling_value_list
u32,date,f64
0,2014-01-01,0.0
1,2014-01-02,0.0
2,2014-01-03,0.0
3,2014-01-04,0.0
4,2014-01-05,0.0
…,…,…
1821,2018-12-27,30.0
1822,2018-12-28,30.0
1823,2018-12-29,30.0
